In [ ]:
from DTC.dtc_executor import DTCExecutor
from DTC.route_skeleton import RouteSkeleton
import json
from math import ceil

In [ ]:
import pickle

#a = DTCExecutor()
#pc = a.create_point_cloud_with_n_points(17000000, city=True)



In [ ]:
#with open('./pc.pickle', 'wb') as handle:
#    pickle.dump(pc, file=handle)

In [ ]:
pc = None 
with open('./pc.pickle', 'rb') as handle:
    pc = pickle.load(handle)

In [ ]:
from DTC.gridsystem import GridSystem
from DTC.construct_safe_area import SafeArea
gs = GridSystem(pc)
avg = 0
iterations = 0
max_radius = 0
median = []
converted_data = {}
anchor = 0
with open("./AllcitySA(1).json", "r") as file:
    data = json.load(file)
    for key, value in data.items():
        max_radius = max(max_radius, value[0])
        median.append(value[0])
        iterations += 1 
        
        clean_key = key.strip('()').split(',')
        tuple_key = tuple(float(num.strip()) for num in clean_key)
        if value[0] > 50:
            value[0] = 50
            anchor = tuple_key
        avg += value[0]
        converted_data[tuple_key] = SafeArea(tuple_key, radius=value[0], cardinality=value[1], confidence_change=1, normalisation_factor=0.5, cardinality_squish=0.15, max_confidence_change=0.15)
median.sort()
print(median[ceil(len(median)/2)], "     ", anchor)
print(avg/iterations, "maximum radius", max_radius)
gs.safe_areas = converted_data

***Loading SafeAreas and Checking AVG radius of SAs***
- Atm our radius is 33.3 cells, which is 167 meters - this is too much


In [ ]:
import copy
print(len(gs.pc.trajectories))

In [ ]:
#extract r
import random
from DTC.clean import CleanTraj
traj_for_test = random.sample(gs.pc.trajectories, 1467)
clean = CleanTraj(gs.safe_areas, gs.initialization_point, pc.min_latitude, pc.min_longitude)



In [ ]:
a = 0
for traj in traj_for_test:
    traj, c = clean.clean(traj=traj)
    a += len(c)
print(a)
l = 0
for t in traj_for_test:
    l += len(t.points)
print(l)

In [ ]:
# use 5% of 52489 - or 5 % of all the points in the collective pointcloud for all trajectories
from math import ceil
amount_to_shift = ceil(l * 0.05)
print(amount_to_shift, " and ", l)

***We use random.sample to get a random set of trajectories every time this runs***
- furthermore, we take 5% of the points and shift them between 1-200m randomly with a random bearing

In [ ]:
from DTC.distance_calculator import DistanceCalculator
list_of_points = []
for traj in traj_for_test:
    for point in traj.points:
        list_of_points.append(point)

points_to_shift = random.sample(list_of_points, amount_to_shift)
print(len(points_to_shift))

In [ ]:
from DTC.distance_calculator import DistanceCalculator
from DTC.point import Point
from datetime import datetime
#Shifting the points twice seem to give me coordinate that does not work - this does not really make any sense

for point in points_to_shift:
    print(point.longitude)
    point_shifted = DistanceCalculator.shift_point_with_bearing(point, random.randint(1,200), random.randint(0,359))
    point.longitude = point_shifted[0]
    point.latitude = point_shifted[1]
    point.noise = True
    point = DistanceCalculator.shift_point_with_bearing(point, random.randint(1,200), DistanceCalculator.EAST)

In [ ]:
from DTC import clean
clean_t = clean.CleanTraj(gs.safe_areas, gs.initialization_point, pc.min_latitude, pc.min_longitude)


In [ ]:

cleaned_points = []
for traj in traj_for_test:
    traj, a = clean_t.clean(traj=traj)
    cleaned_points.extend(a)


- After Cleaning, make comparison with the old trajectory points

In [ ]:
print(len(cleaned_points))
i = 0
a = 0
for point in cleaned_points:
    a += 1
    if point == True:
        i += 1
print(a, " " , i)
i = 0
for point in list_of_points:
    if point.noise == True:
        i += 1
print(i)

**Test Setup**

- Using the setup from the paper DTC
- Using 1472 trajectories as in the paper in the first set of experiments
- 



**This is only with a MR with 2million points**

*first run*
- 926 cleaned
- 3107 to clean

*second run*
- 1140 cleaned
- 2564 to clean


**Running with 11 million points**

*first run*
- 739 cleaned
- 2573 to clean

*second run*
- 775 cleaned
- 2342 to clean

*third run*
- 850 cleaned
- 2394 to clean

*fourth run*
- 656 cleaned
- 2386 to clean

*fifth run*
- 1137 cleaned
- 3998 to clean

- VERY low amount of true positives - thats bad


In [ ]:
average_cleaned = sum([739, 775, 850, 656]) / sum([2573, 2342, 2394, 2386])
print(average_cleaned)

In [ ]:
from pyproj import Proj, transform

def setup_grid_to_geo_transformer(origin_latitude, origin_longitude):
    # Create a Proj instance for the local grid system. This could be UTM or any other fitting your local context.
    # Here, I'm using a simple Transverse Mercator projection centered on the origin with a specific false easting and northing.
    grid_proj = Proj(proj='tmerc', lat_0=origin_latitude, lon_0=origin_longitude, k=1, x_0=0, y_0=0, ellps='WGS84', units='m')

    # WGS84 Proj instance for lat/long
    geo_proj = Proj(proj='latlong', datum='WGS84')

    return grid_proj, geo_proj

def convert_grid_to_geo(x, y, grid_proj, geo_proj):
    # Transform from grid coordinates to geographic coordinates
    lon, lat = transform(grid_proj, geo_proj, x, y)
    return lat, lon

# Example usage
origin_latitude = 39.915  # Adjust to your grid's origin latitude
origin_longitude = 116.404  # Adjust to your grid's origin longitude
grid_proj, geo_proj = setup_grid_to_geo_transformer(origin_latitude, origin_longitude)

# Example grid coordinates
x, y = 100, 200  # These should be in meters if your grid system uses meters
lat, lon = convert_grid_to_geo(x, y, grid_proj, geo_proj)
print(f"Latitude: {lat}, Longitude: {lon}")